In [1]:
import os
import pickle
import pandas as pd

# ——— 1) EFS üzerindeki cache klasörünün tam yolunu ayarla ———
# Eğer notebook’un root’u user-default-efs ise:
base_dir  = "/home/sagemaker-user/user-default-efs"
cache_dir = os.path.join(base_dir, "weather_cache")
pkl_path  = os.path.join(cache_dir, "last_all_years_cleaned.pkl")

# 2) Dosyanın varlığını kontrol et
if not os.path.exists(pkl_path):
    raise FileNotFoundError(f"Dosya bulunamadı: {pkl_path}")

# 3) Pickle’dan DataFrame’i yükle
with open(pkl_path, "rb") as f:
    df = pickle.load(f)

# 4) Temel kontroller
print("► Boyut (satır, sütun):", df.shape)
print("\n► Sütun tipleri:")
print(df.dtypes)

print("\n► Eksik değer sayıları (sıfırdan büyük olanlar):")
print(df.isnull().sum().loc[lambda x: x>0])

# 5) Sayısal sütunlara genel bakış
num_df = df.select_dtypes(include=["number"])
print("\n► Sayısal sütunların betimleyici istatistikleri:")
print(num_df.describe().T)

# 6) Kategorik sütunlara genel bakış
cat_df = df.select_dtypes(include=["object","category"])
print("\n► Kategorik sütunların betimleyici istatistikleri:")
print(cat_df.describe().T)


► Boyut (satır, sütun): (248817, 75)

► Sütun tipleri:
Year                   int64
Month                  int64
DayofMonth             int64
DayOfWeek              int64
FlightDate    datetime64[ns]
                   ...      
u10                  float64
v10                  float64
d2m                  float64
sp                   float64
tp                   float64
Length: 75, dtype: object

► Eksik değer sayıları (sıfırdan büyük olanlar):
Series([], dtype: int64)

► Sayısal sütunların betimleyici istatistikleri:
                                       count          mean          std  \
Year                                248817.0   2004.316795     9.222385   
Month                               248817.0      6.495698     3.446735   
DayofMonth                          248817.0     15.711929     8.780253   
DayOfWeek                           248817.0      3.938417     1.990795   
DepDel15                            248817.0      0.166568     0.372591   
DepartureDelayGroups     

In [2]:
if 'ArrDelayMinutes' in df.columns:
    print("DepMinute sütunu mevcut.")
else:
    print("DepMinute sütunu mevcut değil.")


DepMinute sütunu mevcut değil.


In [3]:
import numpy as np
import pandas as pd

# 1) Saat/dakika sütunlarını çıkar (önce datetime’dan ayıklamış olmanız gerek)
#    Eğer daha önce ayıklamadıysanız:
df['Dep_dt'] = pd.to_datetime(df['DepTime_time'], format='%H:%M:%S', errors='coerce')
df['DepHour']   = df['Dep_dt'].dt.hour
df['DepMinute'] = df['Dep_dt'].dt.minute

df['Arr_dt'] = pd.to_datetime(df['ArrTime_time'], format='%H:%M:%S', errors='coerce')
df['ArrHour']   = df['Arr_dt'].dt.hour
df['ArrMinute'] = df['Arr_dt'].dt.minute

# 2) Zaman fraksiyonu ve döngüsel enkodlama
for prefix in ['Dep','Arr']:
    frac = (df[f'{prefix}Hour']*60 + df[f'{prefix}Minute'])/(24*60)
    df[f'{prefix}Time_frac'] = frac
    df[f'{prefix}_sin']       = np.sin(2*np.pi*frac)
    df[f'{prefix}_cos']       = np.cos(2*np.pi*frac)

# 3) İsterseniz artık bu ara sütunları kaldırın
df.drop(columns=['DepTime_time','ArrTime_time','Dep_dt','Arr_dt',
                 'DepHour','DepMinute','ArrHour','ArrMinute',
                 f'DepTime_frac','ArrTime_frac'], inplace=True)

# 4) Son kontrol
print(df[['Dep_sin','Dep_cos','Arr_sin','Arr_cos']].head())


    Dep_sin   Dep_cos   Arr_sin       Arr_cos
0 -0.984808  0.173648 -0.898794  4.383711e-01
1  0.186524 -0.982450 -0.477159 -8.788171e-01
2 -0.976296 -0.216440 -1.000000 -1.836970e-16
3 -0.300706  0.953717 -1.000000 -1.836970e-16
4 -0.909961  0.414693 -0.288196  9.575714e-01


In [4]:
print("► Boyut (satır, sütun):", df.shape)
print("\n► Sütun tipleri:")
print(df.dtypes)

print("\n► Eksik değer sayıları (sıfırdan büyük olanlar):")
print(df.isnull().sum().loc[lambda x: x>0])

# 5) Sayısal sütunlara genel bakış
num_df = df.select_dtypes(include=["number"])
print("\n► Sayısal sütunların betimleyici istatistikleri:")
print(num_df.describe().T)

# 6) Kategorik sütunlara genel bakış
cat_df = df.select_dtypes(include=["object","category"])
print("\n► Kategorik sütunların betimleyici istatistikleri:")
print(cat_df.describe().T)


► Boyut (satır, sütun): (248817, 77)

► Sütun tipleri:
Year                   int64
Month                  int64
DayofMonth             int64
DayOfWeek              int64
FlightDate    datetime64[ns]
                   ...      
tp                   float64
Dep_sin              float64
Dep_cos              float64
Arr_sin              float64
Arr_cos              float64
Length: 77, dtype: object

► Eksik değer sayıları (sıfırdan büyük olanlar):
Series([], dtype: int64)

► Sayısal sütunların betimleyici istatistikleri:
                                       count          mean          std  \
Year                                248817.0   2004.316795     9.222385   
Month                               248817.0      6.495698     3.446735   
DayofMonth                          248817.0     15.711929     8.780253   
DayOfWeek                           248817.0      3.938417     1.990795   
DepDel15                            248817.0      0.166568     0.372591   
DepartureDelayGroups     

In [5]:
from sklearn.preprocessing import StandardScaler

# —— 1) Bellekteki df’den tüm sayısal sütunları bul —— 
all_num_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()

# —— 2) Hedef sütunumuz (ArrDelay_log_signed) ve döngüsel sin/cos sütunlarını çıkart —— 
to_remove = ['ArrDelay_log_signed','Dep_sin','Dep_cos','Arr_sin','Arr_cos']
num_cols = [c for c in all_num_cols if c not in to_remove]

print("Ölçeklenebilecek sayısal sütunlar:\n", num_cols)

# —— 3) İstediğimiz sütunları seçelim —— 
# (İsterseniz num_cols listesinden defiltre ederek devam edin; örneğin WheelsOff_missing gibi 0/1 sütunlarını bırakmayabilirsiniz)
to_scale = [
    "Distance", "Speed",
    "DepDelayMinutes_log", "Distance_log",
    "ElapsedTimeDiff_log_signed",
    "NetDelayImpact_log_signed",
    "DelayEfficiency_log_signed_winsor2",
    "Latitude", "Longitude",
    "t2m", "u10", "v10", "d2m", "sp", "tp"
]

# —— 4) Ölçekleme —— 
scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[to_scale] = scaler.fit_transform(df[to_scale])

# —— 5) Kontrol edelim —— 
print(df_scaled[to_scale].head())


Ölçeklenebilecek sayısal sütunlar:
 ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepDel15', 'DepartureDelayGroups', 'ArrDel15', 'ArrivalDelayGroups', 'Cancelled', 'Diverted', 'Distance', 'WheelsOff_missing', 'WheelsOn_missing', 'EarlyArrivalImpact', 'Speed', 'DepDelayMinutes_log', 'Distance_log', 'ElapsedTimeDiff_log_signed', 'DelayEfficiency_log_signed_winsor2', 'NetDelayImpact_log_signed', 'Latitude', 'Longitude', 't2m', 'u10', 'v10', 'd2m', 'sp', 'tp']
   Distance     Speed  DepDelayMinutes_log  Distance_log  \
0 -0.859049 -1.036666            -0.683731     -1.042850   
1 -0.186669  0.795688            -0.683731      0.163073   
2 -0.801116  0.168060             0.963046     -0.881342   
3 -0.148047 -0.229045            -0.683731      0.207281   
4  2.955785  0.772187            -0.683731      1.896364   

   ElapsedTimeDiff_log_signed  NetDelayImpact_log_signed  \
0                   -1.004157                  -1.110073   
1                   -0.896238                  -0.752114  

In [6]:
import numpy as np

# 1) Halihazırdaki df’inize erişin (pickle’dan veya bellekte zaten yüklüyse direkt df)
# df = ...

# 2) Döngüsel sin/cos kodlamayı yapalım
df['Month_sin']      = np.sin(2 * np.pi * df['Month']      / 12)
df['Month_cos']      = np.cos(2 * np.pi * df['Month']      / 12)
df['Dow_sin']        = np.sin(2 * np.pi * (df['DayOfWeek'] - 1) / 7)
df['Dow_cos']        = np.cos(2 * np.pi * (df['DayOfWeek'] - 1) / 7)

# (İsteğe bağlı) Ayın gününü de döngüsel kodlayabilirsiniz:
df['Dom_sin']        = np.sin(2 * np.pi * (df['DayofMonth'] - 1) / 31)
df['Dom_cos']        = np.cos(2 * np.pi * (df['DayofMonth'] - 1) / 31)

# 3) Orijinallerini silelim
df.drop(columns=['Month','DayOfWeek','DayofMonth'], inplace=True)

# 4) Binary bayrakları olduğu gibi tutuyoruz:
#    ['Cancelled','Diverted','EarlyArrivalImpact']

# 5) Kontrol edelim
print(df[['Month_sin','Month_cos','Dow_sin','Dow_cos','Dom_sin','Dom_cos']].head())


   Month_sin  Month_cos   Dow_sin  Dow_cos  Dom_sin  Dom_cos
0        0.5   0.866025 -0.781831  0.62349      0.0      1.0
1        0.5   0.866025 -0.781831  0.62349      0.0      1.0
2        0.5   0.866025 -0.781831  0.62349      0.0      1.0
3        0.5   0.866025 -0.781831  0.62349      0.0      1.0
4        0.5   0.866025 -0.781831  0.62349      0.0      1.0


In [7]:
print("► Boyut (satır, sütun):", df.shape)
print("\n► Sütun tipleri:")
print(df.dtypes)

print("\n► Eksik değer sayıları (sıfırdan büyük olanlar):")
print(df.isnull().sum().loc[lambda x: x>0])

# 5) Sayısal sütunlara genel bakış
num_df = df.select_dtypes(include=["number"])
print("\n► Sayısal sütunların betimleyici istatistikleri:")
print(num_df.describe().T)

# 6) Kategorik sütunlara genel bakış
cat_df = df.select_dtypes(include=["object","category"])
print("\n► Kategorik sütunların betimleyici istatistikleri:")
print(cat_df.describe().T)


► Boyut (satır, sütun): (248817, 80)

► Sütun tipleri:
Year                       int64
FlightDate        datetime64[ns]
OriginCityName            object
DestCityName              object
DepDel15                 float64
                       ...      
Month_cos                float64
Dow_sin                  float64
Dow_cos                  float64
Dom_sin                  float64
Dom_cos                  float64
Length: 80, dtype: object

► Eksik değer sayıları (sıfırdan büyük olanlar):
Series([], dtype: int64)

► Sayısal sütunların betimleyici istatistikleri:
                                       count          mean          std  \
Year                                248817.0   2004.316795     9.222385   
DepDel15                            248817.0      0.166568     0.372591   
DepartureDelayGroups                248817.0      0.047987     1.808910   
ArrDel15                            248817.0      0.193773     0.395254   
ArrivalDelayGroups                  248817.0     -0.0923

In [8]:
# Bellekteki df üzerinden FlightDate var mı kontrolü
print("FlightDate sütunu var mı?:", 'DestCityName' in df.columns)


FlightDate sütunu var mı?: True


In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# ——— 1) Bellekteki df’den hedef ve X’i ayır, FlightDate’ı çıkar ———
# (Öncesinde df zaten yüklü ve temizlenmiş olmalı)
y = df["ArrDelay_log_signed"]
X = df.drop(columns=["ArrDelay_log_signed", "FlightDate"])

# ——— 2) Train/Test split ———
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ——— 3) Manual K-Fold Target Encoding ———
high_card = ["OriginCityName", "DestCityName"]
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 3a) Boş DataFrame (aynı index’lerle) hazırla
X_train_te = pd.DataFrame(index=X_train.index, columns=high_card, dtype=float)

# 3b) Her fold’da kendi içinden smoothing’li ortalama hesapla
for tr_idx, val_idx in kf.split(X_train):
    X_tr, y_tr = X_train.iloc[tr_idx], y_train.iloc[tr_idx]
    global_mean = y_tr.mean()

    for col in high_card:
        means  = y_tr.groupby(X_tr[col]).mean()
        counts = X_tr[col].value_counts()
        smooth = counts / (counts + 10)

        idx_val  = X_train.index[val_idx]
        cats_val = X_train.loc[idx_val, col]

        m_vals = cats_val.map(means).fillna(global_mean)
        s_vals = cats_val.map(smooth).fillna(0.0)

        X_train_te.loc[idx_val, col] = s_vals * m_vals + (1 - s_vals) * global_mean

# 3c) Test set’e aynı smoothing’li encoding’i uygula
X_test_te = pd.DataFrame(index=X_test.index, columns=high_card, dtype=float)
global_mean = y_train.mean()

for col in high_card:
    means  = y_train.groupby(X_train[col]).mean()
    counts = X_train[col].value_counts()
    smooth = counts / (counts + 10)

    cats_test = X_test[col]
    m_vals = cats_test.map(means).fillna(global_mean)
    s_vals = cats_test.map(smooth).fillna(0.0)
    X_test_te[col] = s_vals * m_vals + (1 - s_vals) * global_mean

# 3d) Orijinal sütunları çıkar, yerine TE’li sütunları ekle
X_train = pd.concat([X_train.drop(columns=high_card), X_train_te], axis=1)
X_test  = pd.concat([X_test.drop(columns=high_card),  X_test_te],  axis=1)

print("Train shape:", X_train.shape)
print("Test  shape:", X_test.shape)

# ——— 4) Sadece numeric sütunları al (datetime/string tipler atlanacak) ———
num_cols    = X_train.select_dtypes(include=[np.number]).columns
X_train_num = X_train[num_cols]
X_test_num  = X_test [num_cols]

# ——— 5) Modeli eğit & değerlendir (log‐ölçekte) ———
model = HistGradientBoostingRegressor(
    max_iter=200,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)
model.fit(X_train_num, y_train)
y_pred_log = model.predict(X_test_num)

rmse_log = np.sqrt(mean_squared_error(y_test, y_pred_log))
mae_log  = mean_absolute_error(y_test, y_pred_log)

print(f"HGB Test RMSE (log‐scale): {rmse_log:.3f}")
print(f"HGB Test MAE  (log‐scale): {mae_log:.3f}")

# ——— 6) Tahminleri orijinal dakikaya çevir ve yeniden değerlendir ———
#   y = log(ArrDelay_minutes + 1)  varsayımıyla:
y_true_min = np.expm1(y_test)
y_pred_min = np.expm1(y_pred_log)

rmse_min = np.sqrt(mean_squared_error(y_true_min, y_pred_min))
mae_min  = mean_absolute_error(y_true_min, y_pred_min)

print(f"HGB Test RMSE (dakika): {rmse_min:.3f}")
print(f"HGB Test MAE  (dakika): {mae_min:.3f}")


Train shape: (199053, 78)
Test  shape: (49764, 78)
HGB Test RMSE (log‐scale): 0.044
HGB Test MAE  (log‐scale): 0.020
HGB Test RMSE (dakika): 8.721
HGB Test MAE  (dakika): 0.656
